## 1. Environment Setup

In [1]:
# Check GPU availability
!nvidia-smi

Wed Jan  7 11:14:43 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 571.96                 Driver Version: 571.96         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
|  0%   30C    P8             10W /  285W |       0MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Install required packages
!pip install ultralytics>=8.1.0 tensorboard Augmentor opencv-python-headless tqdm PyYAML -q

print("\n Packages installed successfully!")


 Packages installed successfully!


In [3]:
# Import libraries
import os
import sys
import torch
import numpy as np
from pathlib import Path
from datetime import datetime

# Check PyTorch and CUDA
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA device: NVIDIA GeForce RTX 4070 Ti SUPER
CUDA version: 12.1


In [4]:

# Set working directory
WORK_DIR = "C:\\Users\\Cerelab\\Desktop\\GroupIJS2\\"
os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)
print(f"Working directory: {os.getcwd()}")

Working directory: C:\Users\Cerelab\Desktop\GroupIJS2


In [4]:
!pip install kagglehub -q

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("anulayakhare/crackathon-data")

print("Path to dataset files:", path)

C:\Users\Cerelab\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 9.90G/9.90G [10:39<00:00, 16.6MB/s]  

Extracting files...


Path to dataset files: C:\Users\Cerelab\.cache\kagglehub\datasets\anulayakhare\crackathon-data\versions\1


In [7]:
%%writefile yolov9_ep/__init__.py
# YOLOv9-EP Pipeline Package
__version__ = "1.0.1"

Writing yolov9_ep/__init__.py


## 2. Configuration

In [33]:
# ============================================================
# CONFIGURATION - Modify these settings for your experiment
# ============================================================

CONFIG = {
    # Dataset paths
    "DATASET_DIR": "C:\\Users\\Cerelab\\.cache\\kagglehub\\datasets\\anulayakhare\\crackathon-data\\versions\\1\\randomized_dataset",  # Your dataset directory
    "OUTPUT_DIR": "C:\\Users\\Cerelab\\Desktop\\GroupIJS2\\outputs",    # Output directory

    # Dataset info
    "NUM_CLASSES": 5,
    "CLASS_NAMES": [
        "Longitudinal_Crack",   # 0
        "Transverse_Crack",     # 1
        "Alligator_Crack",      # 2
        "Other_Corruption",     # 3
        "Pothole"               # 4
    ],

    # Training hyperparameters
    "EPOCHS": 80,
    "BATCH_SIZE": 16,            # Reduced for Colab GPU memory
    "IMG_SIZE": 768,
    "OPTIMIZER": "SGD",
    "LR0": 0.01,                # Initial learning rate
    "LRF": 0.0001,                # Final LR factor
    "MOMENTUM": 0.937,
    "WEIGHT_DECAY": 0.0005,
    "WARMUP_EPOCHS": 3.0,

    # Deterministic training
    "SEED": 42,
    "DETERMINISTIC": True,

    # Inference thresholds
    "CONF_THRESHOLD": 0.15,     # Confidence threshold
    "IOU_THRESHOLD": 0.55,      # NMS IoU threshold (high for EP)

    # TTA settings for EP
    "TTA_ENABLED": True,
    "TTA_SCALES": [0.67, 0.83, 1.0],
    "TTA_FLIP": True,
    "TTA_SHARPEN": True,
    "TTA_NOISE": True,

    # Model
    "PRETRAINED_WEIGHTS": "yolov9c.pt",
    "DEVICE": "0",  # GPU index

    # Experiment name
    "EXPERIMENT_NAME": f"yolov9c_ep_{datetime.now().strftime('%Y%m%d_%H%M%S')}",

    # Checkpoint settings
    "SAVE_PERIOD": 2,          # Save checkpoint every N epochs
    "BEST_METRIC": "mAP50-95",  # Metric for best model: "mAP50" or "mAP50-95"
}

# Create output directories
os.makedirs(CONFIG["OUTPUT_DIR"], exist_ok=True)
os.makedirs(os.path.join(CONFIG["OUTPUT_DIR"], "runs"), exist_ok=True)
os.makedirs(os.path.join(CONFIG["OUTPUT_DIR"], "tensorboard"), exist_ok=True)
os.makedirs(os.path.join(CONFIG["OUTPUT_DIR"], "checkpoints"), exist_ok=True)

print("="*60)
print("CONFIGURATION LOADED")
print("="*60)
print(f"\n📋 Dataset: {CONFIG['NUM_CLASSES']} Classes")
for i, name in enumerate(CONFIG["CLASS_NAMES"]):
    print(f"   {i} → {name}")
print(f"\n⚙️ Training Settings:")
print(f"   Epochs: {CONFIG['EPOCHS']}")
print(f"   Batch Size: {CONFIG['BATCH_SIZE']}")
print(f"   Image Size: {CONFIG['IMG_SIZE']}")
print(f"   Optimizer: {CONFIG['OPTIMIZER']}")
print(f"\n📊 Best Model Selection: {CONFIG['BEST_METRIC']}")
print("="*60)

CONFIGURATION LOADED

📋 Dataset: 5 Classes
   0 → Longitudinal_Crack
   1 → Transverse_Crack
   2 → Alligator_Crack
   3 → Other_Corruption
   4 → Pothole

⚙️ Training Settings:
   Epochs: 80
   Batch Size: 16
   Image Size: 768
   Optimizer: SGD

📊 Best Model Selection: mAP50-95


In [6]:
# ============================================================
# CHECKPOINT MANAGEMENT - Resume Training Support (FIXED)
# ============================================================

import json
import logging

# Setup logging for debugging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

CHECKPOINT_DIR = os.path.join(CONFIG["OUTPUT_DIR"], "checkpoints")
DRIVE_CHECKPOINT_DIR = "/content/drive/MyDrive/yolov9_ep_checkpoints"

def find_latest_checkpoint():
    """Find the latest checkpoint to resume training."""
    checkpoints = []

    # Check local runs directory - prioritize LAST checkpoint (most recent training)
    runs_dir = os.path.join(CONFIG["OUTPUT_DIR"], "runs")
    logger.info(f"Checking for checkpoints in: {runs_dir}")
    
    if os.path.exists(runs_dir):
        for exp_dir in sorted(Path(runs_dir).iterdir(), reverse=True):
            if exp_dir.is_dir():
                last_pt = exp_dir / "weights" / "last.pt"
                if last_pt.exists():
                    mtime = last_pt.stat().st_mtime
                    checkpoints.append((str(last_pt), mtime))
                    logger.info(f"  Found: {last_pt}")

    # Check Google Drive checkpoints
    if os.path.exists(DRIVE_CHECKPOINT_DIR):
        logger.info(f"Checking Google Drive: {DRIVE_CHECKPOINT_DIR}")
        for f in sorted(Path(DRIVE_CHECKPOINT_DIR).glob("*.pt"), reverse=True):
            mtime = f.stat().st_mtime
            checkpoints.append((str(f), mtime))
            logger.info(f"  Found: {f.name}")

    if checkpoints:
        checkpoints.sort(key=lambda x: x[1], reverse=True)
        latest = checkpoints[0][0]
        logger.info(f"Latest checkpoint selected: {latest}")
        return latest
    
    logger.info("No checkpoints found")
    return None

def load_training_state():
    """Load previous training state if exists."""
    state_path = os.path.join(CONFIG["OUTPUT_DIR"], "training_state.json")
    if os.path.exists(state_path):
        try:
            with open(state_path, 'r') as f:
                state = json.load(f)
                logger.info(f"Loaded training state from: {state_path}")
                return state
        except Exception as e:
            logger.warning(f"Failed to load training state: {e}")
            return None
    return None

def verify_checkpoint(checkpoint_path):
    """Verify checkpoint file integrity."""
    if not os.path.exists(checkpoint_path):
        logger.error(f"Checkpoint not found: {checkpoint_path}")
        return False
    
    file_size = os.path.getsize(checkpoint_path) / (1024 * 1024)
    if file_size < 1:
        logger.error(f"Checkpoint file too small ({file_size:.2f} MB): {checkpoint_path}")
        return False
    
    logger.info(f"Checkpoint verified: {checkpoint_path} ({file_size:.2f} MB)")
    return True

# Check for existing checkpoints
RESUME_CHECKPOINT = find_latest_checkpoint()
PREVIOUS_STATE = load_training_state()

print("\n" + "="*60)
print("CHECKPOINT STATUS")
print("="*60)

if RESUME_CHECKPOINT and verify_checkpoint(RESUME_CHECKPOINT):
    print(f"✅ Found valid checkpoint: {RESUME_CHECKPOINT}")
    if PREVIOUS_STATE:
        print(f"   Experiment: {PREVIOUS_STATE.get('experiment_name', 'Unknown')}")
        print(f"   Previous epochs: {PREVIOUS_STATE.get('epochs_completed', 'Unknown')}")
        metrics = PREVIOUS_STATE.get('metrics', {})
        print(f"   Best mAP@0.5: {metrics.get('mAP50', 0):.4f}")
        print(f"   Best mAP@0.5:0.95: {metrics.get('mAP50-95', 0):.4f}")
    print(f"\n   To resume: Set RESUME_TRAINING = True below")
else:
    print("📝 No valid checkpoint found. Training will start fresh.")
    RESUME_CHECKPOINT = None

print("="*60)

# ============================================================
# SET THIS TO RESUME TRAINING
# ============================================================
RESUME_TRAINING = True  # ← Change to True to resume from checkpoint

INFO:__main__:Checking for checkpoints in: C:\Users\Cerelab\Desktop\GroupIJS2\outputs\runs
INFO:__main__:  Found: C:\Users\Cerelab\Desktop\GroupIJS2\outputs\runs\yolov9c_ep_20260103_112944\weights\last.pt
INFO:__main__:Latest checkpoint selected: C:\Users\Cerelab\Desktop\GroupIJS2\outputs\runs\yolov9c_ep_20260103_112944\weights\last.pt
INFO:__main__:Checkpoint verified: C:\Users\Cerelab\Desktop\GroupIJS2\outputs\runs\yolov9c_ep_20260103_112944\weights\last.pt (98.13 MB)



CHECKPOINT STATUS
✅ Found valid checkpoint: C:\Users\Cerelab\Desktop\GroupIJS2\outputs\runs\yolov9c_ep_20260103_112944\weights\last.pt

   To resume: Set RESUME_TRAINING = True below


## 3. Dataset Preparation

In [7]:
import yaml
from glob import glob
from collections import Counter

def validate_dataset(dataset_dir):
    """Validate dataset structure and class distribution."""
    results = {
        "train_images": len(glob(os.path.join(dataset_dir, "train/images/*"))),
        "train_labels": len(glob(os.path.join(dataset_dir, "train/labels/*.txt"))),
        "val_images": len(glob(os.path.join(dataset_dir, "val/images/*"))),
        "val_labels": len(glob(os.path.join(dataset_dir, "val/labels/*.txt"))),
        "test_images": len(glob(os.path.join(dataset_dir, "test/images/*"))),
    }

    print("📊 Dataset Statistics:")
    print(f"  Train: {results['train_images']} images, {results['train_labels']} labels")
    print(f"  Val: {results['val_images']} images, {results['val_labels']} labels")
    print(f"  Test: {results['test_images']} images (no labels - for submission)")

    # Analyze class distribution in training set
    print("\n📈 Class Distribution (Training Set):")
    class_counts = Counter()
    label_files = glob(os.path.join(dataset_dir, "train/labels/*.txt"))

    for label_file in label_files:
        with open(label_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    class_counts[class_id] += 1

    total_annotations = sum(class_counts.values())
    for class_id in range(CONFIG["NUM_CLASSES"]):
        count = class_counts.get(class_id, 0)
        pct = (count / total_annotations * 100) if total_annotations > 0 else 0
        bar = "█" * int(pct / 2)
        print(f"   {class_id} ({CONFIG['CLASS_NAMES'][class_id]:<20}): {count:>5} ({pct:>5.1f}%) {bar}")

    print(f"\n   Total annotations: {total_annotations}")

    return results

def create_data_yaml(dataset_dir, output_path, nc, names):
    """Create data.yaml for YOLO training."""
    data_config = {
        'path': os.path.abspath(dataset_dir),
        'train': 'train/images',
        'val': 'val/images',
        'test': 'test/images',
        'nc': nc,
        'names': names
    }

    with open(output_path, 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)

    print(f"\n✅ Created data.yaml at: {output_path}")
    print(f"   Classes: {nc}")
    print(f"   Names: {names}")
    return output_path

# Validate dataset

# Create data.yaml
DATA_YAML = os.path.join(CONFIG["OUTPUT_DIR"], "data.yaml")
create_data_yaml(
    CONFIG["DATASET_DIR"],
    DATA_YAML,
    CONFIG["NUM_CLASSES"],
    CONFIG["CLASS_NAMES"]
)


✅ Created data.yaml at: C:\Users\Cerelab\Desktop\GroupIJS2\outputs\data.yaml
   Classes: 5
   Names: ['Longitudinal_Crack', 'Transverse_Crack', 'Alligator_Crack', 'Other_Corruption', 'Pothole']


'C:\\Users\\Cerelab\\Desktop\\GroupIJS2\\outputs\\data.yaml'

## 4. Training YOLOv9-Compact

In [10]:
import random
import numpy as np
import torch

def set_seed(seed=42):
    """Set random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f"🎲 Random seed set to {seed}")

# Set seed for deterministic training
if CONFIG["DETERMINISTIC"]:
    set_seed(CONFIG["SEED"])

🎲 Random seed set to 42


In [8]:
from ultralytics import YOLO

# Load pretrained YOLOv9-Compact
print(f"Loading pretrained weights: {CONFIG['PRETRAINED_WEIGHTS']}")
model = YOLO(CONFIG["PRETRAINED_WEIGHTS"])

print(f"\n✅ Model loaded successfully!")
print(f"Model type: YOLOv9-Compact")

Loading pretrained weights: yolov9c.pt

✅ Model loaded successfully!
Model type: YOLOv9-Compact


In [34]:
# Training configuration - mAP-based best model selection
TRAIN_ARGS = {
    'data': DATA_YAML,
    'epochs': CONFIG["EPOCHS"],
    'batch': CONFIG["BATCH_SIZE"],
    'imgsz': CONFIG["IMG_SIZE"],

    # Optimizer settings (SGD)
    'optimizer': CONFIG["OPTIMIZER"],
    'lr0': CONFIG["LR0"],
    'lrf': CONFIG["LRF"],
    'momentum': CONFIG["MOMENTUM"],
    'weight_decay': CONFIG["WEIGHT_DECAY"],
    'warmup_epochs': CONFIG["WARMUP_EPOCHS"],        
    'cos_lr': True,         # Better LR decay than linear
    'amp': True, 

    # Deterministic training
    'seed': CONFIG["SEED"],
    'deterministic': CONFIG["DETERMINISTIC"],

    # Output settings
    'project': os.path.join(CONFIG["OUTPUT_DIR"], "runs"),
    'name': CONFIG["EXPERIMENT_NAME"],
    'exist_ok': True,

    # Training settings
    'patience': 30,
    'device': CONFIG["DEVICE"],
    'workers': 6,

    # Checkpoint settings - saves best based on mAP (default behavior)
    'save': True,
    'save_period': CONFIG["SAVE_PERIOD"],  # Save every N epochs

    # Augmentation (NO rotation to preserve orientation)
    'degrees': 0.0,
    'shear': 0.0,
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.0,
    'close_mosaic': 10,

    # Validation
    'val': True,
    'plots': True,
}

print("Training Configuration:")
print("-"*40)
for key, value in TRAIN_ARGS.items():
    print(f"  {key}: {value}")
print("-"*40)
print(f"📊 Best model will be saved based on: mAP@0.5:0.95")
print(f"💾 Checkpoints saved every {CONFIG['SAVE_PERIOD']} epochs")

Training Configuration:
----------------------------------------
  data: C:\Users\Cerelab\Desktop\GroupIJS2\outputs\data.yaml
  epochs: 80
  batch: 16
  imgsz: 768
  optimizer: SGD
  lr0: 0.01
  lrf: 0.0001
  momentum: 0.937
  weight_decay: 0.0005
  warmup_epochs: 3.0
  cos_lr: True
  amp: True
  seed: 42
  deterministic: True
  project: C:\Users\Cerelab\Desktop\GroupIJS2\outputs\runs
  name: yolov9c_ep_20260107_151027
  exist_ok: True
  patience: 30
  device: 0
  workers: 6
  save: True
  save_period: 2
  degrees: 0.0
  shear: 0.0
  flipud: 0.0
  fliplr: 0.5
  mosaic: 1.0
  mixup: 0.0
  close_mosaic: 10
  val: True
  plots: True
----------------------------------------
📊 Best model will be saved based on: mAP@0.5:0.95
💾 Checkpoints saved every 2 epochs


In [10]:
# ==========================================
# START TRAINING (FIXED - with proper resume)
# ==========================================
import shutil

print("\n" + "="*60)
print("🚀 STARTING TRAINING")
print("="*60 + "\n")

# Determine resume mode
should_resume = RESUME_TRAINING and RESUME_CHECKPOINT

if should_resume:
    print(f"📂 RESUMING from checkpoint: {RESUME_CHECKPOINT}")
    print(f"   Verifying checkpoint integrity...")
    
    try:
        model = YOLO(RESUME_CHECKPOINT)
        TRAIN_ARGS['resume'] = True
        # Remove conflicting args when resuming
        if 'exist_ok' in TRAIN_ARGS:
            TRAIN_ARGS['exist_ok'] = True  # Allow resume to overwrite
        print(f"   ✅ Checkpoint loaded successfully")
    except Exception as e:
        print(f"   ❌ Failed to load checkpoint: {e}")
        print(f"   Starting fresh instead...")
        model = YOLO(CONFIG["PRETRAINED_WEIGHTS"])
        TRAIN_ARGS['resume'] = True
else:
    print("🆕 Starting FRESH training")
    print(f"   Loading pretrained weights: {CONFIG['PRETRAINED_WEIGHTS']}")
    try:
        model = YOLO(CONFIG["PRETRAINED_WEIGHTS"])
        TRAIN_ARGS['resume'] = False
    except Exception as e:
        print(f"❌ Failed to load pretrained model: {e}")
        raise

print(f"\n{'='*60}")

# Train with error handling for checkpoint recovery
training_completed = False
try:
    logger.info(f"Training arguments: {TRAIN_ARGS}")
    results = model.train(**TRAIN_ARGS)
    training_completed = True
    print("\n" + "="*60)
    print("✅ TRAINING COMPLETE")
    print("="*60)

except KeyboardInterrupt:
    training_completed = False
    print("\n" + "="*60)
    print("⚠️ TRAINING INTERRUPTED BY USER")
    print("="*60)
    print("✅ Checkpoint saved automatically. To resume:")
    print("  1. Set RESUME_TRAINING = True in checkpoint cell")
    print("  2. Re-run from checkpoint cell onwards")
    
except RuntimeError as e:
    if "CUDA" in str(e) or "out of memory" in str(e):
        print("\n" + "="*60)
        print("❌ GPU/Memory Error")
        print("="*60)
        print(f"Error: {e}")
        print("Solutions:")
        print("  • Reduce BATCH_SIZE in CONFIG")
        print("  • Reduce IMG_SIZE")
        print("  • Clear GPU cache: torch.cuda.empty_cache()")
        print("  • Set RESUME_TRAINING=True to continue later")
    raise

except Exception as e:
    training_completed = False
    print(f"\n❌ Training error: {e}")
    print("Check checkpoint was saved for resume.")
    logger.exception("Training failed with exception:")
    raise

INFO:__main__:Training arguments: {'data': 'C:\\Users\\Cerelab\\Desktop\\GroupIJS2\\outputs\\data.yaml', 'epochs': 80, 'batch': 16, 'imgsz': 768, 'optimizer': 'SGD', 'lr0': 0.01, 'lrf': 0.0001, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'cos_lr': True, 'amp': True, 'seed': 42, 'deterministic': True, 'project': 'C:\\Users\\Cerelab\\Desktop\\GroupIJS2\\outputs\\runs', 'name': 'yolov9c_ep_20260107_111459', 'exist_ok': True, 'patience': 30, 'device': '0', 'workers': 6, 'save': True, 'save_period': 2, 'degrees': 0.0, 'shear': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'close_mosaic': 10, 'val': True, 'plots': True, 'resume': True}



🚀 STARTING TRAINING

📂 RESUMING from checkpoint: C:\Users\Cerelab\Desktop\GroupIJS2\outputs\runs\yolov9c_ep_20260103_112944\weights\last.pt
   Verifying checkpoint integrity...
   ✅ Checkpoint loaded successfully

New https://pypi.org/project/ultralytics/8.3.248 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.243  Python-3.12.1 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=C:\Users\Cerelab\Desktop\GroupIJS2\outputs\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_

In [11]:
# ============================================================
# TRAINING RESULTS & CHECKPOINT SAVING (mAP-based)
# ============================================================

if 'results' in dir() and results is not None:
    # Extract metrics
    precision = results.results_dict.get('metrics/precision(B)', 0)
    recall = results.results_dict.get('metrics/recall(B)', 0)
    mAP50 = results.results_dict.get('metrics/mAP50(B)', 0)
    mAP50_95 = results.results_dict.get('metrics/mAP50-95(B)', 0)
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    print("\n" + "="*60)
    print("📊 FINAL TRAINING METRICS")
    print("="*60)
    print(f"  Precision:      {precision:.4f}")
    print(f"  Recall:         {recall:.4f}")
    print(f"  F1-Score:       {f1:.4f}")
    print(f"  mAP@0.5:        {mAP50:.4f}  ← Primary metric")
    print(f"  mAP@0.5:0.95:   {mAP50_95:.4f}  ← Best model selection")
    print("="*60)

    # Get model paths
    BEST_MODEL = os.path.join(results.save_dir, "weights", "best.pt")
    LAST_MODEL = os.path.join(results.save_dir, "weights", "last.pt")

    print(f"\n📁 Model Locations:")
    print(f"  Best (mAP): {BEST_MODEL}")
    print(f"  Last:       {LAST_MODEL}")

    # Save mAP-based checkpoint with metrics in filename
    def save_map_checkpoint(model_path, mAP50, mAP50_95, checkpoint_dir):
        """Save checkpoint with mAP scores in filename."""
        if not os.path.exists(model_path):
            print(f"⚠️ Model not found: {model_path}")
            return None

        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        checkpoint_name = f"yolov9c_mAP50_{mAP50:.4f}_mAP50-95_{mAP50_95:.4f}_{timestamp}.pt"
        checkpoint_path = os.path.join(checkpoint_dir, checkpoint_name)

        shutil.copy(model_path, checkpoint_path)
        print(f"\n💾 mAP checkpoint saved: {checkpoint_path}")

        # Backup to Google Drive
        if os.path.exists("/content/drive/MyDrive"):
            os.makedirs(DRIVE_CHECKPOINT_DIR, exist_ok=True)
            drive_path = os.path.join(DRIVE_CHECKPOINT_DIR, checkpoint_name)
            shutil.copy(model_path, drive_path)
            print(f"☁️ Backed up to Drive: {drive_path}")

        return checkpoint_path

    # Save best model checkpoint
    MAP_CHECKPOINT = save_map_checkpoint(BEST_MODEL, mAP50, mAP50_95, CHECKPOINT_DIR)

    # Save training state for resume
    training_state = {
        'experiment_name': CONFIG["EXPERIMENT_NAME"],
        'epochs_completed': CONFIG["EPOCHS"],
        'best_model': BEST_MODEL,
        'last_model': LAST_MODEL,
        'map_checkpoint': MAP_CHECKPOINT,
        'metrics': {
            'precision': float(precision),
            'recall': float(recall),
            'f1': float(f1),
            'mAP50': float(mAP50),
            'mAP50-95': float(mAP50_95)
        },
        'config': {k: str(v) if not isinstance(v, (int, float, bool, list)) else v
                   for k, v in CONFIG.items()},
        'timestamp': datetime.now().isoformat()
    }

    state_path = os.path.join(CONFIG["OUTPUT_DIR"], "training_state.json")
    with open(state_path, 'w') as f:
        json.dump(training_state, f, indent=2)
    print(f"\n📋 Training state saved: {state_path}")

    # Also save to Drive
    if os.path.exists("/content/drive/MyDrive"):
        drive_state_path = os.path.join(DRIVE_CHECKPOINT_DIR, "training_state.json")
        with open(drive_state_path, 'w') as f:
            json.dump(training_state, f, indent=2)

else:
    print("⚠️ No training results available. Check if training completed.")


📊 FINAL TRAINING METRICS
  Precision:      0.6625
  Recall:         0.6052
  F1-Score:       0.6326
  mAP@0.5:        0.6489  ← Primary metric
  mAP@0.5:0.95:   0.3537  ← Best model selection

📁 Model Locations:
  Best (mAP): C:\Users\Cerelab\Desktop\GroupIJS2\outputs\runs\yolov9c_ep_20260103_112944\weights\best.pt
  Last:       C:\Users\Cerelab\Desktop\GroupIJS2\outputs\runs\yolov9c_ep_20260103_112944\weights\last.pt

💾 mAP checkpoint saved: C:\Users\Cerelab\Desktop\GroupIJS2\outputs\checkpoints\yolov9c_mAP50_0.6489_mAP50-95_0.3537_20260107_121404.pt

📋 Training state saved: C:\Users\Cerelab\Desktop\GroupIJS2\outputs\training_state.json
